In [5]:
import pulp
from pulp import *
import numpy as np
import pandas as pd
import math

## Single machine problem

In [41]:
batches = list(range(10))
quantities = {'Vanilla':4,
              'Coffee': 1,
              'Chocolate Chip': 2,
              'Massachusetts Mud': 3}
flavors = list(quantities.keys())

A_array = np.array([[1,0,1,0],
                    [0,1,0,1],
                    [0,0,1,0],
                    [0,0,0,1]])
A = pd.DataFrame({}, columns = flavors)
for r in range(A_array.shape[0]):
    A.loc[len(A.index)] = A_array[r]
A.index = flavors

In [43]:
# Initialize model
model = LpProblem('Minimize Single Machine Cleaning', sense= LpMinimize)

# Define decision variables
# Flavor indicator
x = LpVariable.dicts('x_', [(i,t) for i in flavors for t in batches], lowBound=None, upBound=None, cat='Binary')
# Cleaning indicator
y = LpVariable.dicts('y_', batches, lowBound=None, upBound=None, cat='Binary')

# Define objective value
model += lpSum([y[t] for t in batches])


# Define constraints
# At most one flavor at a time
for t in batches:
    model += lpSum([x[(i, t)] for i in flavors]) <= 1
    
# Fill order quantities
for i in flavors:
    model += lpSum([x[(i, t)] for t in batches]) >= quantities[i]

# Clean machine if needed
for i in flavors:
    for t in batches[1:]:
        model += lpSum([A.loc[i,j]*x[(j,t)] for j in flavors]) + y[t] >= x[i,t-1] 

In [44]:
model.solve()
print('Status:', LpStatus[model.status])
print()

for t in batches:
    flavor_idx = [x[(i,t)].varValue for i in flavors].index(1.0)
    print('Batch #{}: {}'.format(t+1, flavors[flavor_idx]))

print()
print('Cleaning count: {}'.format(value(model.objective)))
print('Cleaning times:', [y[(t)].varValue for t in batches])


Status: Optimal

Batch #1: Vanilla
Batch #2: Vanilla
Batch #3: Vanilla
Batch #4: Vanilla
Batch #5: Chocolate Chip
Batch #6: Chocolate Chip
Batch #7: Coffee
Batch #8: Massachusetts Mud
Batch #9: Massachusetts Mud
Batch #10: Massachusetts Mud

Cleaning count: 1.0
Cleaning times: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]


## Multi-machine problem

In [92]:
machines = list(range(2))
quantities = {'Vanilla':4,
              'Coffee': 3,
              'Chocolate Chip': 5,
              'Massachusetts Mud': 3}
flavors = list(quantities.keys())

batches_needed = math.ceil((sum(quantities.values())/len(machines)))
batches = list(range(int(batches_needed)))

A_array = np.array([[1,1,1,1],
                    [0,1,0,1],
                    [0,0,1,0],
                    [0,0,0,1]])
A = pd.DataFrame({}, columns = flavors)
for r in range(A_array.shape[0]):
    A.loc[len(A.index)] = A_array[r]
A.index = flavors


In [105]:
# Initialize model
model = LpProblem('Minimize Single Machine Cleaning', sense= LpMinimize)

# Define decision variables
# Flavor indicator
x = LpVariable.dicts('x_', [(i,m,t) for i in flavors for m in machines for t in batches], lowBound=None, upBound=None, cat='Binary')
# Cleaning indicator
y = LpVariable.dicts('y_', [(m,t) for m in machines for t in batches], lowBound=None, upBound=None, cat='Binary')
# Flavor count for a machine
z = LpVariable.dicts('z_', machines, lowBound=0, upBound=None, cat='Integer')

# Define objective value
model += lpSum([y[m,t] for m in machines for t in batches]) + lpSum([z[m] for m in machines])

# Define constraints
# At most one flavor at a time
for t in batches:
    for m in machines:
        model += lpSum([x[(i, m, t)] for i in flavors]) <= 1
    
# Fill order quantities
for i in flavors:
    model += lpSum([x[(i, m, t)] for m in machines for t in batches]) == quantities[i]

# Clean machine if needed
for i in flavors:
    for m in machines:
        for t in batches[1:]:
            model += lpSum([A.loc[i,j]*x[(j,m,t)] for j in flavors]) + y[m,t] >= x[i,m,t-1]

# Count of flavors per machine
for m in machines:
    model += lpSum([x[(i, m, t)] for i in flavors for t in batches]) == z[m]
    

In [109]:
model.solve()
print('Status:', LpStatus[model.status])
for m in machines:
    print('\nMACHINE {}'.format(m+1))
    print('Cleaning count:', sum([y[(m,t)].varValue for t in batches]))
    spot = 1
    for t in batches:
        if y[(m,t)].varValue == 1.0:
            print('--> CLEAN MACHINE')
        try:
            flavor_idx = [x[(i,m,t)].varValue for i in flavors].index(1.0)
            print('--> Batch #{}: {}'.format(spot, flavors[flavor_idx]))
            spot += 1
        except ValueError:
            pass
            #print('--> Batch #{}: None'.format(t+1))

print('\nTotal cleaning count: {}'.format(sum([y[(m,t)].varValue for m in machines for t in batches])))
#print('Objective value: {}'.format(value(model.objective)))

Status: Optimal

MACHINE 1
Cleaning count: 0.0
--> Batch #1: Vanilla
--> Batch #2: Coffee
--> Batch #3: Coffee
--> Batch #4: Coffee
--> Batch #5: Massachusetts Mud
--> Batch #6: Massachusetts Mud
--> Batch #7: Massachusetts Mud

MACHINE 2
Cleaning count: 0.0
--> Batch #1: Vanilla
--> Batch #2: Vanilla
--> Batch #3: Vanilla
--> Batch #4: Chocolate Chip
--> Batch #5: Chocolate Chip
--> Batch #6: Chocolate Chip
--> Batch #7: Chocolate Chip
--> Batch #8: Chocolate Chip

Total cleaning count: 0.0
